In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys
sys.path.append(".")

base_url_node_1 = "http://204.12.168.157:8000"

from epochs import (
    EpochGroup,
    GENESIS_GUARDIAN_ADDRESSES,
)

epoch_groups_chain = [EpochGroup.load(base_url_node_1, "current")]

epoch_group = epoch_groups_chain[0]
while True:
    prev_epoch_group = EpochGroup.load(base_url_node_1, epoch_group.epoch_id - 1)
    epoch_groups_chain.append(prev_epoch_group)
    epoch_group = prev_epoch_group
    if epoch_group.epoch_id == 1:
        break

    # if len(epoch_groups_chain) > 6:
    #     break



In [14]:
def validate_signers_total_consensus_weight(
    epoch_group_next: EpochGroup,
    epoch_group: EpochGroup,
):
    signers_addresses = epoch_group_next.created_at_block.signers_addresses
    signers_total_consensus_weight = epoch_group.signers_total_consensus_weight(signers_addresses)
    total_consensus_weight = epoch_group.get_total_consensus_weight()
    return float(signers_total_consensus_weight) / float(total_consensus_weight)

def validate_first_epoch_group(
    epoch_group: EpochGroup,
):
    return True

In [15]:
for pos, epoch_group in enumerate(epoch_groups_chain):
    if pos == len(epoch_groups_chain) - 1:
        break
    if epoch_group.epoch_id == 1:
        print("Validating first epoch group")
        validate_first_epoch_group(epoch_group)
        break

    epoch_group_prev = epoch_groups_chain[pos + 1]

    validated_by_weight_portion = validate_signers_total_consensus_weight(epoch_group, epoch_group_prev)
    print(f"Validating epoch group {epoch_group_prev.epoch_id} from {epoch_group.epoch_id}: signed by {validated_by_weight_portion} of the total weight")


Validating epoch group 68 from 69: signed by 0.8821017539588152 of the total weight
Validating epoch group 67 from 68: signed by 0.9837354147304065 of the total weight
Validating epoch group 66 from 67: signed by 0.9985771284068559 of the total weight
Validating epoch group 65 from 66: signed by 1.0 of the total weight
Validating epoch group 64 from 65: signed by 0.8971826481739249 of the total weight
Validating epoch group 63 from 64: signed by 0.9953318314292033 of the total weight
Validating epoch group 62 from 63: signed by 0.998253177034269 of the total weight
Validating epoch group 61 from 62: signed by 0.9928822798922423 of the total weight
Validating epoch group 60 from 61: signed by 0.9287303154957143 of the total weight
Validating epoch group 59 from 60: signed by 0.8463587106254018 of the total weight
Validating epoch group 58 from 59: signed by 0.9989807985894597 of the total weight
Validating epoch group 57 from 58: signed by 0.9861205876510368 of the total weight
Validati

-----